# Analyse de données d'amplicons 16S avec QIIME 2

Ce notebook a pour but de réaliser une analyse complète de données de séquençage d'amplicons 16S en utilisant l'écosystème QIIME 2. Les étapes incluent le calcul de la diversité alpha, l'analyse de la diversité bêta, la composition taxonomique et une analyse de redondance basée sur la distance (db-RDA).


## 1. Configuration de l'environnement

Cette section installe QIIME 2 et les dépendances nécessaires. Exécutez ces cellules si vous utilisez Google Colab.


In [ ]:
# Exécutez ce script pour installer QIIME 2
%run setup_qiime2

# Installation des plugins et autres dépendances
!pip install scikit-bio
!pip install seaborn
!qiime plugins install --url https://github.com/qiime2/q2-longitudinal/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-breakaway/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-emperor/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-feature-classifier/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-taxa/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-diversity/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-gneiss/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-composition/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-sample-classifier/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-deicode/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-songbird/archive/master.zip
!qiime plugins install --url https://github.com/qiime2/q2-longitudinal/archive/master.zip


## 2. Importation et description des données

Dans cette section, nous allons charger les fichiers de données et de métadonnées en tant qu'artefacts QIIME 2.


In [ ]:
import qiime2 as q2
from qiime2.plugins import demux, metadata, feature_table, diversity, emperor

# Charger les métadonnées
metadata_path = 'metadata.tsv'
metadata = q2.Metadata.load(metadata_path)

# Charger la table de contingence
table_path = 'table-emp.qza'
table = q2.Artifact.load(table_path)

# Charger les séquences représentatives
rep_seqs_path = 'rep-seqs.qza'
rep_seqs = q2.Artifact.load(rep_seqs_path)

# Charger la taxonomie
taxo_path = 'taxo-silva128.qza'
taxonomy = q2.Artifact.load(taxo_path)

# Afficher un résumé de la table de contingence
!qiime feature-table summarize --i-table {table_path} --o-visualization table.qzv --m-sample-metadata-file {metadata_path}
!qiime metadata tabulate --m-input-file table.qzv --o-visualization table.qzv


## 3. Analyse de la diversité Alpha (indice de Shannon)

Nous allons maintenant calculer l'indice de diversité de Shannon pour chaque échantillon.


In [ ]:
# Calcul de la diversité alpha (Shannon)
!qiime diversity alpha --i-table {table_path} --p-metric shannon --o-alpha-diversity shannon.qza

# Visualisation des résultats
!qiime diversity alpha-group-significance --i-alpha-diversity shannon.qza --m-metadata-file {metadata_path} --o-visualization shannon_significance.qzv


## 4. Analyse de la diversité Bêta (PCoA)

Cette section est dédiée à l'analyse de la diversité bêta via une Analyse en Coordonnées Principales (PCoA) sur une matrice de distance de Bray-Curtis.


In [ ]:
# Calcul de la matrice de distance (Bray-Curtis)
!qiime diversity beta --i-table {table_path} --p-metric braycurtis --o-distance-matrix bray_curtis_distance_matrix.qza

# Analyse PCoA
!qiime diversity pcoa --i-distance-matrix bray_curtis_distance_matrix.qza --o-pcoa pcoa_results.qza

# Visualisation avec Emperor
!qiime emperor plot --i-pcoa pcoa_results.qza --m-metadata-file {metadata_path} --o-visualization pcoa_emperor.qzv


## 5. Composition Taxonomique

Visualisation de la composition taxonomique des échantillons.


In [ ]:
# Création du barplot taxonomique
!qiime taxa barplot --i-table {table_path} --i-taxonomy {taxo_path} --m-metadata-file {metadata_path} --o-visualization taxa_barplot.qzv


### Heatmap de la composition taxonomique (par classe)


In [ ]:
# Agréger la table au niveau de la classe
!qiime taxa collapse --i-table {table_path} --i-taxonomy {taxo_path} --p-level 3 --o-collapsed-table table-l3.qza

# Créer le heatmap
!qiime feature-table heatmap --i-table table-l3.qza --m-metadata-file {metadata_path} --m-metadata-column body-site --p-normalize --o-visualization heatmap-l3.qzv


## 6. Analyse de Redondance basée sur la Distance (db-RDA)

Cette analyse permet d'évaluer la relation entre la composition des communautés microbiennes et les variables environnementales numériques.


In [ ]:
# Assurez-vous que votre fichier de métadonnées contient des colonnes numériques.
# Par exemple, 'ph', 'temperature', etc.
# La commande suivante suppose que 'ph' est une variable dans votre metadata.tsv

# Exécution de la db-RDA
!qiime diversity adonis --i-distance-matrix bray_curtis_distance_matrix.qza --m-metadata-file {metadata_path} --p-formula "ph" --o-visualization adonis_results.qzv

# Note : Remplacez "ph" par la ou les variables numériques pertinentes de votre fichier de métadonnées.
# Pour plusieurs variables, utilisez la syntaxe "var1+var2+..."
